Source: https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html<br>
References: [1](https://learnopencv.com/camera-calibration-using-opencv/), [2](https://medium.com/analytics-vidhya/camera-calibration-with-opencv-f324679c6eb7https://medium.com/analytics-vidhya/camera-calibration-with-opencv-f324679c6eb7)
## Goal
In this section, we will learn about

* types of distortion caused by cameras
* how to find the intrinsic and extrinsic properties of a camera
* how to undistort images based off these properties

$camera \; matrix = \left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]$

In [1]:
import numpy as np
import cv2 as cv
import glob

In [2]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [3]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
BOARD = (6, 7)
objp = np.zeros((BOARD[0]*BOARD[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:BOARD[0], 0:BOARD[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('./sample_calibration_images/Image*.*')

In [8]:
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(
        gray,
        BOARD,
        cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(
            gray, 
            corners,
            (11,11),
            (-1,-1),
            criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, BOARD, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(0)
cv.destroyAllWindows()

## Calibration
Now that we have our object points and image points, we are ready to go for calibration. We can use the function, [cv.calibrateCamera()](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d) which returns the camera matrix, distortion coefficients, rotation and translation vectors etc.

In [5]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [6]:
from tempfile import TemporaryFile
outfile = TemporaryFile()

In [9]:
with open("c_calib.npz", "wb") as file:
    np.savez(file, mtx, dist, rvecs, tvecs)